<a href="https://colab.research.google.com/github/Cutie-tee/nlp_project/blob/main/nlp_textclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the Data

In [4]:
import pandas as pd
#Load the dataset
data =pd.read_csv('/content/training_data_lowercase.csv')
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/training_data_lowercase.csv'

Data Preprocessing

In [ ]:
#Clean the text to convert to lowercase, remove punctuation, whitespace , stop words and then lemmatise

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
stop_words =set(stopwords.words('english'))
lemmatizer =WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatise
    text =' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return text
    # Apply the preprocessing function to the 'text' column
data['text'] = data['text'].apply(preprocess_text)